# Preparação

In [ ]:
# Bibliotecas
import pandas as pd
from pandas import DataFrame
import numpy as np
import sys
import chardet
import chardet.universaldetector
!pip -q install unidecode
import unidecode
from functools import lru_cache
import os
from os import remove
import gc
!pip -q install wget
import wget
!pip -q install py7zr
from py7zr import SevenZipFile
import glob
from io import StringIO

# BigQuery
!pip install --upgrade -q google-cloud-bigquery

# Cloud Storage
!pip install --upgrade -q google-cloud-storage

from google.colab import auth
from google.cloud import storage

# Configurando GCS

In [2]:
# Autenticar
auth.authenticate_user()

In [ ]:
# Inicializa o cliente do GCS
client = storage.Client(project="<seu-projeto-aqui>")
bucket_name = '<seu-bucket-aqui>'
bucket = client.get_bucket(bucket_name)

## Funções

In [6]:
def salvar_no_gcs(arquivo_ou_df, path_blob):
    try:
        # Se for um DataFrame, salva como CSV temporário
        if isinstance(arquivo_ou_df, DataFrame):
            temp_file = "temp.csv"
            arquivo_ou_df.to_csv(temp_file, index=False)
        elif isinstance(arquivo_ou_df, str) and os.path.exists(arquivo_ou_df):
            temp_file = arquivo_ou_df
        else:
            raise ValueError("Entrada deve ser um DataFrame ou caminho de arquivo existente.")

        # Envia para o GCS
        blob = bucket.blob(path_blob)
        blob.upload_from_filename(temp_file)
        print(f"📁 Salvo em: gs://{bucket_name}/{path_blob}")

    except Exception as e:
        print("❌ Erro ao salvar no GCS:", e)

    finally:
        # Remove apenas o temporário criado internamente
        if 'temp_file' in locals() and temp_file == "temp.csv" and os.path.exists(temp_file):
            os.remove(temp_file)

In [7]:
# Função para baixar e extrair os arquivos da CAGED
def extract_caged (ano, mes):
  """ Faz o download dos arquivos do NOVO CAGED para o ano e o mês especificados.
  O período do NOVO CAGED tem início em Janeiro de 2020.
  Nesse script não extraímos os microdados antriores a esse período devido a divergências metodológicas estabelecidas pelo MTE.

  São 3 tipos de arquivo: CAGEDMOV, CAGEDFOR e CAGEDEXC.
    1. CAGEDMOV: movimentações declaradas dentro do prazo com competência de declaração igual a AAAAMM.
    2. CAGEDFOR: movimentações declaradas fora do prazo com competência de declaração igual a AAAAMM.
    3. CAGEDEXC: movimentações excluídas com competência de declaração da exclusão igual a AAAAMM.

  Argumentos:
      ano: Ano do arquivo.
      mes: Mês do arquivo.
  """
  # Download
  if ano < 2020:
        print(f"❌ Microdados de {mes}/{ano} ignorados.")
        return

  tipos_arquivos = ['CAGEDEXC', 'CAGEDFOR', 'CAGEDMOV']
  mes_str = str(mes).zfill(2)

  for tipo in tipos_arquivos:
    url = f'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO CAGED/{ano}/{ano}{mes_str}/{tipo}{ano}{mes_str}.7z'
    nome_arquivo = f'{tipo}{ano}{mes_str}.7z'

    try:
        print(f'⬇️ Baixando {tipo} de {mes_str}/{ano}...')
        wget.download(url, nome_arquivo)
    except Exception as e:
        print(f'❌ Arquivo {tipo} de {mes_str}/{ano} indisponível: {e}')
        continue

    # Extração do arquivo 7z
    try:
        with SevenZipFile(nome_arquivo, mode='r') as archive:
          archive.extractall()
          print(f'✅ Extraído: {tipo}{ano}{mes_str}.txt')
    except Exception as e:
        print(f'❌ Erro na extração: {nome_arquivo} → {e}')
        continue

    # Salvando arquivo extraído no GCS
    try:
        nome_extraido = f'{tipo}{ano}{mes_str}.txt'
        path_gcs = f"caged/raw/ano={ano}/mes={mes_str}/{nome_extraido}"
        salvar_no_gcs(nome_extraido, path_gcs)
    except Exception as e:
        print(f'❌ Erro ao enviar para GCS: {e}')
        continue

    # Remoção do arquivo .7z e .txt após upload
    try:
        remove(nome_arquivo)
        remove(nome_extraido)
        print(f'🗑️ Arquivos locais removidos.\n')
    except Exception as e:
        print(f'⚠️ Erro ao remover arquivos locais: {e}')

# Download e Extração dos arquivos

## Extração (2020 até mais recente)

In [ ]:
# Extraindo arquivos para todo o período
for ano in range(2020, 2026):
    for mes in range(1, 13):
        extract_caged(ano, mes)

## Extração (personalizada)

In [ ]:
# Extraindo arquivos de um mês específico
extract_caged(2024, 8)